In [1]:
import numpy as np
from scipy.integrate import solve_ivp

In [2]:
# Problem 1
def f(t, y):
    return -3.0 * y * np.sin(t)

def ytrue(t):
    return (np.pi/np.sqrt(2.0)) * np.exp(3.0*(np.cos(t)-1.0))

y0 = np.pi/np.sqrt(2.0)
dt_list = [2.0**(-k) for k in range(2, 9)]

errs_fe = []
A1 = None
for dt in dt_list:
    t = np.arange(0.0, 5.0 + dt, dt)
    y = np.empty_like(t)
    y[0] = y0
    for j in range(len(t)-1):
        y[j+1] = y[j] + dt * f(t[j], y[j])
    errs_fe.append(np.mean(np.abs(ytrue(t) - y)))
    if np.isclose(dt, 2.0**-8):
        A1 = y.reshape(-1, 1)

coef_fe = np.polyfit(np.log(dt_list), np.log(errs_fe), 1)
A2 = np.array(errs_fe)
A3 = float(coef_fe[0])

errs_heun = []
A4 = None
for dt in dt_list:
    t = np.arange(0.0, 5.0 + dt, dt)
    y = np.empty_like(t)
    y[0] = y0
    for j in range(len(t)-1):
        k1 = f(t[j], y[j])
        yp = y[j] + dt * k1
        k2 = f(t[j] + dt, yp)
        y[j+1] = y[j] + 0.5 * dt * (k1 + k2)
    errs_heun.append(np.mean(np.abs(ytrue(t) - y)))
    if np.isclose(dt, 2.0**-8):
        A4 = y.reshape(-1, 1)

coef_heun = np.polyfit(np.log(dt_list), np.log(errs_heun), 1)
A5 = np.array(errs_heun)
A6 = float(coef_heun[0])


In [3]:
# Problem 2
def vdp(t, Y, eps):
    y, yp = Y
    return [yp, -eps*(y*y - 1.0)*yp - y]

t = np.arange(0.0, 32.5, 0.5)
eps_list = [0.1, 1.0, 20.0]
A7 = np.column_stack([solve_ivp(lambda tt, YY: vdp(tt, YY, e), (t[0], t[-1]), [np.sqrt(3.0), 1.0], t_eval=t).y[0] for e in eps_list])

tols = np.array([10.0**(-k) for k in range(4, 11)])

def slope_for(method):
    dts = []
    for tol in tols:
        if method is None:
            sol = solve_ivp(lambda tt, YY: vdp(tt, YY, 1.0), (0.0, 32.0), [2.0, np.pi**2], atol=tol, rtol=tol)
        else:
            sol = solve_ivp(lambda tt, YY: vdp(tt, YY, 1.0), (0.0, 32.0), [2.0, np.pi**2], atol=tol, rtol=tol, method=method)
        dts.append(np.mean(np.diff(sol.t)))
    coef = np.polyfit(np.log(dts), np.log(tols), 1)
    return float(coef[0])

A8 = slope_for(None)
A9 = slope_for("RK23")
A10 = slope_for("LSODA")

In [4]:
# Problem 3
a1, a2, b, c, I = 0.05, 0.25, 0.01, 0.01, 0.1
t = np.arange(0.0, 100.5, 0.5)
y0 = [0.1, 0.1, 0.0, 0.0]

def rhs(t, Y, d12, d21):
    v1, v2, w1, w2 = Y
    dv1 = -v1**3 + (1.0 + a1)*v1**2 - a1*v1 - w1 + I + d12*v2
    dw1 = b*v1 - c*w1
    dv2 = -v2**3 + (1.0 + a2)*v2**2 - a2*v2 - w2 + I + d21*v1
    dw2 = b*v2 - c*w2
    return [dv1, dv2, dw1, dw2]

pairs = [(0.0,0.0), (0.0,0.2), (-0.1,0.2), (-0.3,0.2), (-0.5,0.2)]
sols = []
for d12, d21 in pairs:
    sol = solve_ivp(lambda tt, YY: rhs(tt, YY, d12, d21), (t[0], t[-1]), y0, t_eval=t, method="BDF")
    sols.append(np.column_stack([sol.y[0], sol.y[1], sol.y[2], sol.y[3]]))

A11, A12, A13, A14, A15 = sols
